In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [65]:
from fastai.vision import *
from fastai.metrics import accuracy


from torch.autograd import Variable
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform

In [3]:
PATH = "/scratch/as3ek/chrc_data_case_preserved_1000_normal/"
sz=512
tfms = get_transforms(do_flip=True, flip_vert=True, max_zoom=1.1)

In [4]:
data = (ImageItemList.from_folder(PATH)
        .split_by_folder()          
        .label_from_folder()
        .transform(tfms, size=sz)
        .databunch(bs=16)
        .normalize(imagenet_stats))

In [5]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    
    def forward(self, x): 
        return torch.cat([self.mp(x), self.ap(x)], 1)
    
    
custom_head = nn.Sequential(
                            AdaptiveConcatPool2d(),
                            Flatten(), 
                            nn.BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                            nn.Dropout(p=0.25),
                            nn.Linear(in_features=4096, out_features=512, bias=True),
                            nn.ReLU(),
                            nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                            nn.Dropout(p=0.5),
                            nn.Linear(in_features=512, out_features=10, bias=True),
                            nn.ReLU(),
                            nn.Linear(in_features=10, out_features=3, bias=True),
                            nn.LogSoftmax()
                           )

In [6]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, custom_head=custom_head)

In [9]:
learn.unfreeze()

In [10]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-3))

epoch,train_loss,valid_loss,accuracy
1,0.216016,0.467687,0.817645


/home/as3ek/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
learn.save('resnet_10_output_1_epoch')

In [32]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*(list(learn.model.children())[:-1] + list(list(learn.model.children())[1])[:-3]))
        
    def forward(self, x):
        x = self.features(x)
        return x

model_10_features = ResNet50Bottom(learn.model)

In [82]:
train_df = data.train_ds.to_df()

In [87]:
bs = 16
data_model_trn = pd.DataFrame(columns = list(range(10)).append('fname'))

dl_iter = iter(learn.data.train_dl)
dl = next(dl_iter)
for index, name in enumerate(train_df['x']):
    if ((index % bs == 0) and (index != 0)):
        dl = next(dl_iter)
    image_batch, label_batch = dl
    i = index % bs
    image = image_batch[i].reshape(1, 3, 512, 512)
    label = label_batch[i].reshape(-1)
    
    output = model_10_features(image)
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().detach().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model_trn = pd.concat([data_model_trn, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

StopIteration: 

In [93]:
data_model_trn.to_csv('resnet_train_10_features.csv', index=False)

In [94]:
valid_df = data.valid_ds.to_df()

In [95]:
bs = 16
data_model_val = pd.DataFrame(columns = list(range(10)).append('fname'))

dl_iter = iter(learn.data.valid_dl)
dl = next(dl_iter)
for index, name in enumerate(valid_df['x']):
    if ((index % bs == 0) and (index != 0)):
        dl = next(dl_iter)
    image_batch, label_batch = dl
    i = index % bs
    image = image_batch[i].reshape(1, 3, 512, 512)
    label = label_batch[i].reshape(-1)
    
    output = model_10_features(image)
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().detach().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model_val = pd.concat([data_model_val, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [96]:
data_model_val.to_csv('')

,0,1,2,3,4,5,6,7,8,9,fname
0,-1.875293,-3.157814,2.965702,10.299607,-2.735940,0.187620,-1.872909,-1.374993,2.451687,-1.095966,valid/Celiac/C04-17_01___2000_2000___500_250.jpg
1,-1.836606,-4.860983,6.493116,2.873093,-0.641450,3.320685,-1.727019,-1.924873,-2.036005,-2.435864,valid/Celiac/C12-61_03___3000_3000___0_250.jpg
2,-2.666274,-2.019515,6.552889,8.146291,-1.098976,5.744137,-3.060744,-1.925482,-4.448660,-5.001292,valid/Celiac/C06-31_02___5000_3000___0_500.jpg
3,-2.406507,-4.266439,6.013614,8.526498,-3.687720,0.694869,-3.923469,-3.493088,-2.600260,-3.566477,valid/Celiac/C12-64_01b___14000_4000___250_250...
4,-3.024812,-3.292944,-7.292481,24.522245,-9.205770,-6.891388,-4.915971,-4.427787,5.271486,-1.345942,valid/Celiac/C10-47_01___0_6000___500_500.jpg
5,-2.676574,-2.862277,2.557347,14.018425,-4.163038,-1.635188,-3.268087,-2.691452,1.096530,-1.784487,valid/Celiac/C10-47_02___8000_5000___500_0.jpg
6,0.138922,0.519861,3.702438,-7.764839,5.711498,7.347148,3.359416,4.328333,-0.357703,-0.073413,valid/Celiac/C06-31_01___6272_1000___0_250.jpg
7,-0.269775,-2.943590,7.708514,0.576766,-0.502597,0.939570,-1.198607,-0.087274,-2.182706,-2.150644,valid/Celiac/C10-47_01___9000_8000___500_500.jpg
8,-3.251037,-4.043772,0.627356,18.858118,-8.763917,-7.112542,-5.840136,-4.358951,1.894303,-1.977742,valid/Celiac/C10-49_02___3000_4000___250_250.jpg
9,-1.947158,2.278532,6.133047,6.361450,-1.229402,0.904040,-2.033902,2.714606,-1.510068,-2.376673,valid/Celiac/C12-64_01b___3000_5000___500_250.jpg
